In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import kart_ocr
import os
from PIL import Image


In [33]:
input_path = "./2023-11-15/"
for r, _, files in os.walk(input_path):
    for f in files:
        path = os.path.join(r, f)
        with Image.open(path) as img:
            display(kart_ocr.read_scores(img, 0.35, 30))


[[([[4, 6], [102, 6], [102, 32], [4, 32]], '9: Pesve'),
  ([[34, 48], [110, 48], [110, 72], [34, 72]], '38.305'),
  ([[34, 88], [110, 88], [110, 114], [34, 114]], '38.056'),
  ([[34, 128], [110, 128], [110, 154], [34, 154]], '37.126'),
  ([[34, 168], [110, 168], [110, 194], [34, 194]], '37.370'),
  ([[34, 208], [106, 208], [106, 234], [34, 234]], '36 971'),
  ([[34, 250], [110, 250], [110, 274], [34, 274]], '36.878'),
  ([[34, 290], [110, 290], [110, 314], [34, 314]], '37.056'),
  ([[34, 330], [110, 330], [110, 356], [34, 356]], '36.778'),
  ([[34, 370], [108, 370], [108, 396], [34, 396]], '36.814'),
  ([[34, 412], [110, 412], [110, 436], [34, 436]], '37.003'),
  ([[34, 452], [108, 452], [108, 476], [34, 476]], '36 471'),
  ([[34, 492], [106, 492], [106, 518], [34, 518]], '36.591'),
  ([[34, 532], [110, 532], [110, 560], [34, 560]], '36.579'),
  ([[34, 572], [106, 572], [106, 598], [34, 598]], '36.671'),
  ([[34, 612], [110, 612], [110, 640], [34, 640]], '36.526'),
  ([[34, 652], [108,

[[([[0, 4], [86, 4], [86, 30], [0, 30]], '2: Raha'),
  ([[28, 44], [104, 44], [104, 70], [28, 70]], '39.135'),
  ([[30, 84], [102, 84], [102, 110], [30, 110]], '38.196'),
  ([[28, 124], [104, 124], [104, 150], [28, 150]], '37.770'),
  ([[30, 164], [104, 164], [104, 190], [30, 190]], '37.695'),
  ([[28, 204], [104, 204], [104, 232], [28, 232]], '37.346'),
  ([[30, 246], [104, 246], [104, 272], [30, 272]], '55.626'),
  ([[28, 286], [104, 286], [104, 312], [28, 312]], '37.363'),
  ([[30, 326], [104, 326], [104, 354], [30, 354]], '38.160'),
  ([[28, 366], [104, 366], [104, 392], [28, 392]], '37.536'),
  ([[28, 408], [102, 408], [102, 432], [28, 432]], '378104'),
  ([[28, 448], [102, 448], [102, 472], [28, 472]], '37.807'),
  ([[28, 488], [104, 488], [104, 514], [28, 514]], '37.628'),
  ([[28, 528], [102, 528], [102, 554], [28, 554]], '37.577'),
  ([[30, 568], [104, 568], [104, 594], [30, 594]], '37.383'),
  ([[28, 610], [104, 610], [104, 634], [28, 634]], '45.185'),
  ([[28, 648], [104, 64

[[([[2, 1], [102, 1], [102, 30], [2, 30]], '4: Henpe'),
  ([[30, 42], [104, 42], [104, 66], [30, 66]], '38.777'),
  ([[30, 80], [104, 80], [104, 106], [30, 106]], '38.586'),
  ([[30, 122], [106, 122], [106, 146], [30, 146]], '37.898'),
  ([[30, 162], [106, 162], [106, 188], [30, 188]], '37.429'),
  ([[30, 202], [104, 202], [104, 228], [30, 228]], '37 312'),
  ([[30, 244], [104, 244], [104, 268], [30, 268]], '37.334'),
  ([[30, 284], [106, 284], [106, 308], [30, 308]], '37.665'),
  ([[30, 324], [106, 324], [106, 348], [30, 348]], '37.395'),
  ([[30, 364], [104, 364], [104, 390], [30, 390]], '36.934'),
  ([[30, 404], [102, 404], [102, 430], [30, 430]], '37.351'),
  ([[30, 444], [104, 444], [104, 472], [30, 472]], '37.156'),
  ([[30, 486], [104, 486], [104, 512], [30, 512]], '37.248'),
  ([[30, 528], [104, 528], [104, 552], [30, 552]], '37.144'),
  ([[30, 568], [104, 568], [104, 592], [30, 592]], '37.092'),
  ([[30, 608], [104, 608], [104, 632], [30, 632]], '37.193'),
  ([[30, 648], [104,

[[([[0, 0], [132, 0], [132, 32], [0, 32]], '9: Hoestrup'),
  ([[30, 44], [104, 44], [104, 68], [30, 68]], '36.958'),
  ([[30, 84], [104, 84], [104, 110], [30, 110]], '37.004'),
  ([[30, 124], [104, 124], [104, 150], [30, 150]], '37.132'),
  ([[32, 164], [104, 164], [104, 190], [32, 190]], '36.196'),
  ([[30, 204], [102, 204], [102, 230], [30, 230]], '36 541'),
  ([[30, 246], [104, 246], [104, 270], [30, 270]], '36.085'),
  ([[30, 286], [102, 286], [102, 312], [30, 312]], '40.727'),
  ([[30, 326], [104, 326], [104, 352], [30, 352]], '35.934'),
  ([[30, 366], [104, 366], [104, 392], [30, 392]], '37.782'),
  ([[30, 406], [104, 406], [104, 432], [30, 432]], '35.845'),
  ([[30, 446], [102, 446], [102, 472], [30, 472]], '35.961'),
  ([[30, 488], [104, 488], [104, 512], [30, 512]], '36.096'),
  ([[30, 528], [104, 528], [104, 556], [30, 556]], '35.798'),
  ([[30, 570], [102, 570], [102, 594], [30, 594]], '36.077'),
  ([[28, 608], [104, 608], [104, 634], [28, 634]], '35.170'),
  ([[28, 650], [1

[[([[0, 4], [94, 4], [94, 30], [0, 30]], '11: Raha'),
  ([[24, 42], [100, 42], [100, 68], [24, 68]], '37.335'),
  ([[26, 84], [98, 84], [98, 108], [26, 108]], '37.233'),
  ([[24, 124], [98, 124], [98, 150], [24, 150]], '37.074'),
  ([[24, 164], [98, 164], [98, 190], [24, 190]], '37.105'),
  ([[24, 204], [98, 204], [98, 230], [24, 230]], '36.852'),
  ([[24, 246], [98, 246], [98, 270], [24, 270]], '36.893'),
  ([[24, 286], [96, 286], [96, 312], [24, 312]], '37.101'),
  ([[24, 326], [98, 326], [98, 352], [24, 352]], '37.532'),
  ([[24, 366], [98, 366], [98, 392], [24, 392]], '37.263'),
  ([[24, 408], [98, 408], [98, 432], [24, 432]], '36.888'),
  ([[24, 448], [98, 448], [98, 474], [24, 474]], '37.202'),
  ([[24, 488], [98, 488], [98, 514], [24, 514]], '36.813'),
  ([[24, 528], [98, 528], [98, 556], [24, 556]], '36.900'),
  ([[24, 570], [98, 570], [98, 594], [24, 594]], '36 932'),
  ([[22, 610], [98, 610], [98, 634], [22, 634]], '37.935'),
  ([[22, 650], [98, 650], [98, 674], [22, 674]], '

[[([[4, 2], [94, 2], [94, 26], [4, 26]], '4: Srran'),
  ([[32, 40], [108, 40], [108, 66], [32, 66]], '39.020'),
  ([[32, 82], [108, 82], [108, 106], [32, 106]], '37.589'),
  ([[32, 122], [106, 122], [106, 148], [32, 148]], '37.121'),
  ([[32, 162], [108, 162], [108, 188], [32, 188]], '37.138'),
  ([[32, 202], [106, 202], [106, 228], [32, 228]], '36.812'),
  ([[34, 244], [108, 244], [108, 268], [34, 268]], '37.003'),
  ([[32, 284], [108, 284], [108, 308], [32, 308]], '36.895'),
  ([[32, 324], [106, 324], [106, 348], [32, 348]], '36.827'),
  ([[32, 364], [106, 364], [106, 390], [32, 390]], '37.178'),
  ([[32, 404], [106, 404], [106, 430], [32, 430]], '37.242'),
  ([[32, 444], [104, 444], [104, 470], [32, 470]], '36.501'),
  ([[32, 484], [106, 484], [106, 510], [32, 510]], '36 816'),
  ([[32, 526], [106, 526], [106, 550], [32, 550]], '36.665'),
  ([[32, 566], [106, 566], [106, 592], [32, 592]], '37.356'),
  ([[32, 606], [106, 606], [106, 632], [32, 632]], '36.417'),
  ([[32, 646], [106, 6